## 모듈 import

In [2]:
#모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pycaret.classification  import *

# feature selection
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

#Scailing
from sklearn.preprocessing import PowerTransformer

#Label Encoding
from sklearn.preprocessing import LabelEncoder

In [3]:
#삡운드
import winsound as sd
# 삡싸운드
def beepsound():
    fr = 2000   # range : 37 ~ 32767
    du = 1000 # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

## data 불러오기

In [4]:
# 데이터가 저장된 경로 설정
data_path = '../daplatformers2021/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'trainset.csv')
test = pd.read_csv(data_path + 'testset_final.csv')
submission = pd.read_csv(data_path + 'sample_submission_final.csv')

In [5]:
train = train.drop(columns=['매니저최초가입일', '매니저최초서비스일', '매니저주소'])

### 전처리

In [6]:
data = pd.concat([train,test]).reset_index(drop=True)

- 결측치 채움

In [7]:
### 결측치 제거

# 결측치 0으로 채움
data['청소교육이수여부'] = data['청소교육이수여부'].fillna(0)
data['CS교육이수여부'] = data['CS교육이수여부'].fillna(0)
data['부재중여부'] = data['부재중여부'].fillna(0)
# 결측치 최빈값으로 채움
data['매니저이동방법'] = data['매니저이동방법'].fillna(data['매니저이동방법'].mode()[0])
data['매니저사용휴대폰'] = data['매니저사용휴대폰'].fillna(data['매니저사용휴대폰'].mode()[0])
data['고객가입일'] = data['고객가입일'].fillna(data['고객가입일'].mode()[0]) #데이터 바뀌고 test에 생긴 에러
data['결재형태'] = data['결재형태'].fillna(data['결재형태'].mode()[0]) #데이터 바뀌고 test에 생긴 에러

data['접수시각'] = data['접수시각'].fillna(data['접수시각'].mode()[0])
data['반려동물'] = data['반려동물'].fillna('없음')
# 결측치 판단하여 없음으로 채움
data['우선청소'] = data['우선청소'].fillna('없음')

### 평수 평균 확인 -> 31.xx -> 30평대로 채움
# (10*len(data.query('평수 == "10평대"')) + 20*len(data.query('평수 == "20평대"')) + 
#  30*len(data.query('평수 == "30평대"')) + 40*len(data.query('평수 == "40평대이상"')))/ \
# (len(data.query('평수 == "10평대"'))+len(data.query('평수 == "20평대"'))+
#  len(data.query('평수 == "30평대"'))+len(data.query('평수 == "40평대이상"')))
data['평수'] = data['평수'].fillna('30평대')

In [8]:
#test와 train의 구조를 맞춰줌
data['매니저생년월일'] = data['매니저생년월일'].astype('str').apply(lambda x: x[0:4])
# 서비스시간 피처 생성
data['서비스시간'] =  (pd.to_datetime(data['서비스시작시간'], format="%H:%M:%S") -\
                        pd.to_datetime(data['서비스종료시간'], format="%H:%M:%S")).astype('str')
# 새로운 피처 만들기
data['매니저나이'] = data['매니저생년월일'].apply(lambda x: 2021-int(x))
data['매니저나이대'] = data['매니저나이'].apply(lambda x: f'{str(x)[0]}0대')
# 고객가입개월수
data['고객가입개월수_str'] = data['고객가입일'].apply(lambda x:str((2021-(int(x[0:4])+1))*12 + 12-int(x[5:7])+1 + 11) )
data['고객가입개월수_int'] = data['고객가입일'].apply(lambda x:(2021-(int(x[0:4])+1))*12 + 12-int(x[5:7])+1 + 11)
# 고객가입년차
def mon_cnt(x):
    x = int(x)
    if x < 12: return '1년미만'
    elif 12 <= x < 24: return '1년차'
    elif 24 <= x < 36: return '2년차'
data['고객가입년차'] = data['고객가입개월수_int'].apply(mon_cnt)
# 두가지의 변수를 곱해줌
data['나이*개월수'] = data['매니저나이']*data['고객가입개월수_int']
#전체회차 / 고객가입개월수_int
data['한달평균사용횟수'] = data['전체회차'] / data['고객가입개월수_int']

In [14]:
# 저장해놓기
data_SEQ = data.SEQ
# 인코딩을 위하여 제거 / 이후 나중에 병합함
y_data = data['매칭성공여부']
data = data.drop(columns = ['SEQ','매칭성공여부'])

In [16]:
#라벨인코딩

#범주형변수 수동으로 분리
data[['장기서비스여부','기존고객여부','부재중여부','쿠폰사용여부','매니저성별',
         'CS교육이수여부','청소교육이수여부','부재중서비스가능여부','추천인여부']
     ] = data[['장기서비스여부','기존고객여부','부재중여부','쿠폰사용여부','매니저성별',
         'CS교육이수여부','청소교육이수여부','부재중서비스가능여부','추천인여부'
     ]].astype('str')

# 범주형 변수와 수치형 변수를 분리
cat_features = data.select_dtypes(include=['object']).columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list() 

#수치형변수 scailing
data[num_features] = PowerTransformer(standardize=True).fit_transform(data[num_features])

#라벨인코딩
#위의 범주형 변수들을 라벨인코딩
for f in cat_features:
    data[f] = LabelEncoder().fit_transform(data[f])

#위에서 떼어놨던 피처 병합
data = pd.concat([data_SEQ, data],axis=1)
data = pd.concat([data, y_data],axis=1);data.head(3)

,SEQ,접수일,접수시각,장기서비스여부,최초서비스일,전체회차,현재회차,서비스일자,서비스시작시간,서비스종료시간,기존고객여부,결재형태,서비스주소,주거형태,평수,고객가입일,반려동물,부재중여부,우선청소,쿠폰사용여부,매니저생년월일,매니저성별,매니저사용휴대폰,매니저이동방법,근무가능지역,CS교육이수여부,청소교육이수여부,부재중서비스가능여부,추천인여부,서비스시간,매니저나이,매니저나이대,고객가입개월수_str,고객가입개월수_int,고객가입년차,나이*개월수,한달평균사용횟수,매칭성공여부
0,T06420,45,379,1,56,-0.650410,0.882026,84,11,3,1,1,48,1,2,6,6,0,3,0,3,0,2,0,3,0,0,1,0,5,1.824058,4,24,1.365244,2,2.015637,-1.156575,0.0
1,T15430,48,379,1,56,-0.131658,0.882026,103,1,6,1,1,48,1,2,18,6,0,3,0,23,0,2,0,6,1,1,1,0,3,-1.237119,2,23,1.230494,2,0.569809,-0.769221,0.0
2,T23790,345,379,1,342,1.998486,1.752481,432,11,3,1,3,48,1,3,202,6,0,3,0,17,0,2,0,6,0,0,0,0,5,-0.458846,3,4,-0.881830,1,-0.876181,1.676526,0.0
3,T18740,337,379,1,332,-0.131658,1.285355,423,11,4,1,1,26,1,2,115,6,0,1,0,3,0,2,0,3,0,0,1,0,4,1.824058,4,13,0.065858,1,0.643857,-0.285810,0.0
4,T08451,182,379,1,198,-1.271044,0.218171,257,11,3,1,1,48,1,2,77,6,0,3,0,13,0,2,0,6,0,0,1,0,5,0.125776,3,18,0.701711,2,0.760175,-1.399370,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27248,T31589,306,379,1,310,1.408811,-1.086998,378,11,3,1,3,48,1,3,119,6,0,1,1,8,0,2,0,6,0,0,1,0,5,0.932147,4,13,0.065858,1,0.411062,0.793248,NaN
27249,T36236,152,379,1,158,-0.131658,-1.086998,198,5,8,0,1,48,1,2,107,6,0,3,0,23,0,2,1,6,0,0,1,0,5,-1.237119,2,14,0.190620,1,-0.193198,-0.351799,NaN
27250,T34729,361,379,1,359,-0.131658,-1.086998,437,11,3,1,3,48,1,3,1,6,0,1,0,8,0,2,0,6,0,0,1,0,5,0.932147,4,24,1.365244,2,1.678505,-0.810634,NaN
27251,T53830,153,379,1,160,1.408811,1.752481,229,11,3,1,1,47,1,2,31,6,0,3,1,11,0,1,1,6,0,0,0,0,5,0.438163,3,22,1.096740,2,1.235516,0.308100,NaN


### AB_test

In [19]:
# 검증
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
# 모델
from sklearn.ensemble import RandomForestClassifier
# 셔플, 시간측정
import random
import time
# skfold
from sklearn.model_selection import StratifiedKFold

In [20]:
#train과 test를 분리
train = data.iloc[:23009,:].drop(columns=['SEQ'])
test = data.iloc[23009:,:].drop(columns = ['매칭성공여부']).reset_index(drop=True)

In [21]:
label = np.array(train['매칭성공여부'])
train = train.drop('매칭성공여부', axis=1)

In [ ]:
#자동화 - RandomForestClassifier
# 반복횟수를 지정한 후 랜덤하게 리스트를 섞어 가장 최적의 스코어가 나오는 피처를 찾음
#####  Straified Kfold  ######
skf = StratifiedKFold(n_splits=5)
data_list = ['매니저생년월일','매니저나이','서비스주소','고객가입일','부재중서비스가능여부',
                '매니저이동방법','근무가능지역','청소교육이수여부','매니저나이대',
                '결재형태','서비스종료시간','CS교육이수여부',
                '전체회차','접수일','서비스일자','매니저사용휴대폰','반려동물','부재중여부',
                '서비스시작시간','추천인여부','서비스시간','우선청소','최초서비스일','주거형태','접수시각',
                 '나이*개월수','한달평균사용횟수' ,'고객가입개월수_int','고객가입개월수_str','고객가입년차','평수','매니저성별',
                 '기존고객여부', '쿠폰사용여부','장기서비스여부','현재회차'
                 ]

final_list = [] # 최종적으로 컬럼들을 확인할 리스트 생성
final_score = []

epoch = 5 # 반복횟수 지정

for e in range(epoch):
    start = time.time() # 시간 측정을 위함
    # 돌때마다 섞어줌
    random.shuffle(data_list)
    data = data[data_list]
    
    column_list = ['매니저생년월일']
    score_list = [0] # 스코어 저장 / 맨처음은 0으로 시작
    
    eopch_score = [0] # 한번 돌때마다의 스코어
    for column in list(data.columns):    
        # 확인을 위한 리스트 생성
        val_list = [ c for c in column_list]
        val_list.append(column)
        data_val = data[val_list]

        #위에서 떼어놨던 피처 병합
        data_val = pd.concat([data_SEQ, data_val, y_data],axis=1)

        #train과 test를 분리
        train = data_val.iloc[:23009,:].drop(columns=['SEQ'])
        test = data_val.iloc[23009:,:].drop(columns = ['매칭성공여부']).reset_index(drop=True)

        # 형식에 맞게 변경
        label = np.array(train['매칭성공여부'])
        train = train.drop('매칭성공여부', axis=1)
        # SKfold
        val_scores = []
        for i, (train_idx, val_idx) in enumerate(skf.split(train, label)):
            x_train, y_train = train.iloc[train_idx, :], label[train_idx,]
            x_val, y_val = train.iloc[val_idx, :], label[val_idx,]

        #     print(i)
        #     print(f'train: {len(x_train)} // valid: {len(x_val)}')
            clf = RandomForestClassifier(random_state=0)

            clf.fit(x_train, y_train)

            y_pred = clf.predict(x_val)

            score = roc_auc_score(y_val, y_pred)

        #     print('Validation roc-auc score: {:.6f}\n'.format(score))

            val_scores.append(score)
        score = np.mean(val_scores)
        # 스코어와 컬럼 추가
        if score_list[-1] < score:
            score_list.append(score)
            if column not in column_list: # 컬럼에 없으면 추가
                column_list.append(column)
    es = score_list[-1]
    el = column_list
    
    # 한번 돌때마다 저장
    final_score.append(es)
    final_list.append(el)
    
    best_score = max(final_score) #스코어의 max값 찾음
    best_columns = final_list[final_score.index(best_score)] #스코어가 max인 값의 피처 찾음
    
    end = time.time() # 시간 측정을 위함
                
    print(f'eopch: {e+1}/{epoch}  score: {score_list[-1]}  time: {end - start}')
    

print(f'\nmax_score: {best_score},\n final_list: {best_columns}')

eopch: 1/5  score: 0.9105279655493351  time: 213.3219563961029


## END